In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import random
import datetime as dt
import requests as rq
import pandas as pd

from job_client_graphql import Client, CellDisplay

## Prerequisite
+ Launch JOB server - see [documentation](TBD) section **5-Examples**

In [3]:
pages = {
    'aggrid': 'https://observablehq.com/@oscar6echo/job-example-aggrid',
}

In [4]:
s = Client(server_url='http://localhost:4001/graphql',
           default_channel='channel-1',
           password='mysecret',
           pages=pages,         
           debug=False)
print(s.build_url('aggrid'))
s.show_btn_page('aggrid')

https://observablehq.com/@oscar6echo/job-example-aggrid?host=localhost&port=4001&path=/graphql&ssl=False&channel=channel-1


In [5]:
url = 'https://raw.githubusercontent.com/ag-grid/ag-grid/master/grid-packages/ag-grid-docs/src/olympicWinnersSmall.json'
data = json.loads(rq.get(url).text)
print(f'nb lines = {len(data)}')


nb lines = 8618


In [6]:
df = pd.DataFrame(data)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8618 entries, 0 to 8617
Data columns (total 10 columns):
age        8613 non-null float64
athlete    8618 non-null object
bronze     8618 non-null int64
country    8618 non-null object
date       8618 non-null object
gold       8618 non-null int64
silver     8618 non-null int64
sport      8618 non-null object
total      8618 non-null int64
year       8618 non-null int64
dtypes: float64(1), int64(5), object(4)
memory usage: 673.4+ KB


## Write

In [7]:
def build_grid_options(df):
    col_defs = [{'field': c} for c in df.columns]
    grid_options = {
        'columnDefs': col_defs,
        'defaultColDef': {
            'editable': True,
            'sortable': True,
            'filter': True,
            'resizable': True
        },
        'rowSelection': 'single',
        'rowData': df.iloc[:100, :].to_dict(orient='records'),
    }
    return grid_options


In [8]:
grid_options = build_grid_options(df)
s.publish_write('grid', json.dumps({'gridOptions': grid_options}))

True

## Subscribe

### DISPLAY

In [9]:
out = CellDisplay(name='subscribe-status', max_lines=8)
show_lines = []

def show(channel, name, value):
    ts = dt.datetime.now().strftime('%H:%M:%S.%f')[:-4]
    s = f'received on {channel}: {name} --> {value} ({ts})'
    out.update(s, kind='text', append=True)


received on channel-1: grid-row --> {"age":24,"athlete":"Aleksey Nemov","bronze":3,"country":"Russia","date":"01/10/2000","gold":2,"silver":1,"sport":"Gymnastics","total":6,"year":2000} (22:56:43.88)
received on channel-1: grid-row --> {"age":27,"athlete":"Ryan Lochte","bronze":1,"country":"United States","date":"12/08/2012","gold":2,"silver":2,"sport":"Swimming","total":5,"year":2012} (22:56:45.50)
received on channel-1: grid-row --> {"age":17,"athlete":"Ian Thorpe","bronze":0,"country":"Australia","date":"01/10/2000","gold":3,"silver":2,"sport":"Swimming","total":5,"year":2000} (22:56:47.24)

In [10]:
out.display()


received on channel-1: grid-row --> {"age":24,"athlete":"Aleksey Nemov","bronze":3,"country":"Russia","date":"01/10/2000","gold":2,"silver":1,"sport":"Gymnastics","total":6,"year":2000} (22:56:43.88)
received on channel-1: grid-row --> {"age":27,"athlete":"Ryan Lochte","bronze":1,"country":"United States","date":"12/08/2012","gold":2,"silver":2,"sport":"Swimming","total":5,"year":2012} (22:56:45.50)
received on channel-1: grid-row --> {"age":17,"athlete":"Ian Thorpe","bronze":0,"country":"Australia","date":"01/10/2000","gold":3,"silver":2,"sport":"Swimming","total":5,"year":2000} (22:56:47.24)

### START

In [11]:
ref = s.subscribe(action=show, test=lambda x: x=='grid-row')

### STOP

In [12]:
s.subscribe_stop(ref)

subscription ref=WT9vNm stopped


## Read

In [13]:
name = 'grid-row'
s.read(name)

{'age': 24,
 'athlete': 'Alicia Coutts',
 'bronze': 1,
 'country': 'Australia',
 'date': '12/08/2012',
 'gold': 1,
 'silver': 3,
 'sport': 'Swimming',
 'total': 5,
 'year': 2012}

## Read w/ Histo

In [14]:
name = 'grid-row'
s.read_histo(name, 4, timestamp=True)

[{'value': {'age': 24,
   'athlete': 'Alicia Coutts',
   'bronze': 1,
   'country': 'Australia',
   'date': '12/08/2012',
   'gold': 1,
   'silver': 3,
   'sport': 'Swimming',
   'total': 5,
   'year': 2012},
  'timestamp': datetime.datetime(2020, 5, 17, 20, 56, 53, 599000)},
 {'value': {'age': 17,
   'athlete': 'Ian Thorpe',
   'bronze': 0,
   'country': 'Australia',
   'date': '01/10/2000',
   'gold': 3,
   'silver': 2,
   'sport': 'Swimming',
   'total': 5,
   'year': 2000},
  'timestamp': datetime.datetime(2020, 5, 17, 20, 56, 47, 246000)},
 {'value': {'age': 27,
   'athlete': 'Ryan Lochte',
   'bronze': 1,
   'country': 'United States',
   'date': '12/08/2012',
   'gold': 2,
   'silver': 2,
   'sport': 'Swimming',
   'total': 5,
   'year': 2012},
  'timestamp': datetime.datetime(2020, 5, 17, 20, 56, 45, 505000)},
 {'value': {'age': 24,
   'athlete': 'Aleksey Nemov',
   'bronze': 3,
   'country': 'Russia',
   'date': '01/10/2000',
   'gold': 2,
   'silver': 1,
   'sport': 'Gymnasti

## Store

In [15]:
s.store

{'channel-1': {'grid-row': {'age': 24,
   'athlete': 'Alicia Coutts',
   'bronze': 1,
   'country': 'Australia',
   'date': '12/08/2012',
   'gold': 1,
   'silver': 3,
   'sport': 'Swimming',
   'total': 5,
   'year': 2012}}}

In [ ]:
s.publish_write('grid', None, add_histo=False)